**Задача**

1. Дообучить русскоязычную модель GPT для генерации фейковых новостей.
2. Датасет необходимо найти самостоятельно.
3. Привести 5-10 примеров сгенерированных фейковых новостей на русском языке.
4. Подобрать параметры генерации, улучшить качество новостей.
5. Изучить model.generate более детально.
6. Добиться
 такой генерации, чтобы не было обрыва мысли, новость заканчивалась либо точкой, либо восклицательным знаком, либо вопросительным (если уместно).

In [2]:
from google.colab import files

# Эта команда откроет диалог выбора файлов — выберите у себя на диске archive.zip
uploaded = files.upload()

Saving archive.zip to archive.zip


In [3]:

!unzip -q archive.zip -d /content/archive


In [4]:
!ls -lh /content/archive

total 38M
-rw-r--r-- 1 root root 7.1M Dec 30  2022 test_bodies.csv
-rw-r--r-- 1 root root 503K Dec 30  2022 test_stances_unlebeledb.csv
-rw-r--r-- 1 root root  29M Dec 30  2022 train_bodies.csv
-rw-r--r-- 1 root root 2.0M Dec 30  2022 train_stances.csv


In [5]:
import pandas as pd

df_train_bodies = pd.read_csv("/content/archive/train_bodies.csv")

texts = df_train_bodies["articleBody"].astype(str).tolist()

# Записываем каждую новость в одну строку файла fake_news_dataset.txt в /content
with open("/content/fake_news_dataset.txt", "w", encoding="utf-8") as f_out:
    for text in texts:
        clean_text = text.replace("\n", " ").strip()
        f_out.write(clean_text + "\n")

# Проверяем, что файл появился
!ls -lh /content/fake_news_dataset.txt


-rw-r--r-- 1 root root 11M Oct  8 11:55 /content/fake_news_dataset.txt


In [6]:
# 1 Импорт и выбор устройства
import torch
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# 2 Загрузка модели и токенизатора
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [8]:
# 3 Подготовка датасета из fake_news_dataset.txt
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="fake_news_dataset.txt",
    block_size=512
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [9]:
# 4 Настройка TrainingArguments
training_args = TrainingArguments(
    output_dir="./finetuned_fake_news",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    report_to="none"
)

In [10]:
# 5 Fine-tuning через Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)
print("=== START TRAINING ===")
trainer.train()
print("=== TRAINING FINISHED ===")

=== START TRAINING ===


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,2.975900
200,2.935800
300,2.886500
400,2.880600
500,2.881800
600,2.869100
700,2.870800
800,2.890900
900,2.842800
1000,2.840800


=== TRAINING FINISHED ===


In [11]:
# 6 Перевод модели в режим генерации
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=3072, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=1024)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=4096, nx=1024)
          (c_proj): Conv1D(nf=1024, nx=4096)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [12]:
# 7 Подготовка промптов и eos_token_id
prompts = [
    "Срочно: в Москве обнаружили",
    "Эксперты заявили, что",
    "Новое исследование показало, что",
    "В социальных сетях появилась информация о том, что",
    "Неожиданно оказалось, что"
]
eos_ids = [
    tokenizer.encode(".", add_special_tokens=False)[0],
    tokenizer.encode("!", add_special_tokens=False)[0],
    tokenizer.encode("?", add_special_tokens=False)[0]
]
max_new_tokens = 80

In [13]:
# 8 Цикл генерации для каждого prompt
for prompt in prompts:
    # 8.1 Токенизировать prompt и перевести на device
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # 8.2 Greedy Search
    out_greedy = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + max_new_tokens,
        do_sample=False,
        eos_token_id=eos_ids,
        early_stopping=True
    )
    text_greedy = tokenizer.decode(out_greedy[0], skip_special_tokens=True)

    # 8.3 Beam Search
    out_beam = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + max_new_tokens,
        num_beams=5,
        early_stopping=True,
        eos_token_id=eos_ids
    )
    text_beam = tokenizer.decode(out_beam[0], skip_special_tokens=True)

    # 8.4 Sampling с temperature и top_p
    out_sample = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + max_new_tokens,
        do_sample=True,
        temperature=1.2,
        top_p=0.9,
        eos_token_id=eos_ids
    )
    text_sample = tokenizer.decode(out_sample[0], skip_special_tokens=True)

    # 8.5 Вывод результатов
    print("="*80)
    print("PROMPT:", prompt)
    print("\n[Greedy]  ", text_greedy)
    print("\n[Beam]    ", text_beam)
    print("\n[Sample]  ", text_sample)
    print("="*80 + "\n")

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


PROMPT: Срочно: в Москве обнаружили

[Greedy]   Срочно: в Москве обнаружили тело бывшего главы «Роснано» Анатолия Чубайса, пропавшего без вести после отравления нервно-паралитическим веществом класса «Новичок». Об этом сообщает «Интерфакс» со ссылкой на источник в экстренных службах.

[Beam]     Срочно: в Москве обнаружили массовое захоронение мирных жителей, убитых российскими военными» — говорится в сообщении.

[Sample]   Срочно: в Москве обнаружили мертвого бывшего губернатора Хабаровского края Сергея Фургала.,
Президент Украины Владимир Зеленский встретился с депутатами украинского парламента и попросил их оказать давление на российские власти, чтобы они выдали российских военнопленных.

PROMPT: Эксперты заявили, что

[Greedy]   Эксперты заявили, что в случае, если Россия не прекратит войну, то через 10-15 лет Россия может полностью исчезнуть с карты мира.

[Beam]     Эксперты заявили, что в ближайшее время в России может начаться массовая вакцинация населения от коронавируса.

[Sa